* PySpark - consultas
*  http://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations
*  https://data-flair.training/blogs/spark-tutorials-home/

In [2]:
?sc.parallelize

In [7]:
from pyspark import SparkConf, SparkContext

In [8]:
print(sc)

<SparkContext master=local[*] appName=PySparkShell>


In [15]:
#Lendo os dados de julho
julRDD = sc.textFile('access_log_Jul95')
julRDD = julRDD.cache()

#Visualizando os dados - julRDD.first()
#'199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'

#Lendo os dados de Agosto
agoRDD = sc.textFile('access_log_Aug95')
agoRDD = agoRDD.cache()

#Visualizando os dados - agoRDD.first()
#'in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839'


In [16]:
# Número de Hosts únicos em Julho
count_julho = julRDD.flatMap(lambda line: line.split(' ')[0]).distinct().count()
print('Número de hosts únicos em Julho: %s' % count_julho)

# Número de Hosts únicos em Agosto
count_agosto = agoRDD.flatMap(lambda line: line.split(' ')[0]).distinct().count()
print('Número de hosts únicos em Agosto %s' % count_agosto)

Número de hosts únicos em Julho: 55
Número de hosts únicos em Agosto 53


In [17]:
# total de erros em Agosto
def total_agosto_404 (y):
    try:
        code_error = y.split(' ')[-2]
        if code_error == '404':
            return True
    except:
        pass
    return False

augustT = agoRDD.filter(total_agosto_404)
print('Erros 404 em Agosto:',augustT.count())

# total de erros em Julho
def total_julho_404(x):
    try:
        code_error = x.split(' ')[-2]
        if code_error == '404':
            return True
    except:
        pass
    return False

julyT = julRDD.filter(total_julho_404)
print('Erros 404 em Julho:',julyT.count())

Erros 404 em Agosto 10056
Erros 404 em Julho 10845


In [20]:
from operator import add

#Total de bytes retornados
def byte_count_total(rdd):
    def countbytes(line):
        try:
            count = int(line.split(" ")[-1])
            if count < 0:
                raise ValueError()
            return count
        except:
            return 0
        
    count = rdd.map(countbytes).reduce(add)
    return count

print('Total de bytes em Julho: %s' % byte_count_total(julRDD))
print('Total de bytes em Agosto: %s' % byte_count_total(agoRDD))

Total de bytes em Julho: 38695973491
Total de bytes em Agosto: 26828341424
